# Extract UMAP embeddings for each plate of CellProfiler features

NOTE: We are using the feature selected profiles per plate.

In [1]:
import glob
import pathlib
import pandas as pd
import numpy as np
import umap

from pycytominer.cyto_utils import infer_cp_features

## Generate Embeddings

### Set constant for whole plates

In [2]:
# Set constants
umap_random_seed = 0
umap_n_components = 2

# Set embeddings directory
output_dir = pathlib.Path("results")
output_dir.mkdir(parents=True, exist_ok=True)

### Set paths to all plates

In [3]:
# Set input path with single-cell profiles
data_dir = pathlib.Path("../3.preprocessing_features/data/single_cell_profiles")

# Select only the feature selected files
file_suffix = "*sc_feature_selected.parquet"

# Obtain file paths for all feature selected plates
fs_files = glob.glob(f"{data_dir}/{file_suffix}")
print(f"There are {len(fs_files)} feature selected files with the following paths:")
fs_files

There are 4 feature selected files with the following paths:


['../3.preprocessing_features/data/single_cell_profiles/localhost240928120001_sc_feature_selected.parquet',
 '../3.preprocessing_features/data/single_cell_profiles/localhost240927060001_sc_feature_selected.parquet',
 '../3.preprocessing_features/data/single_cell_profiles/localhost240927120001_sc_feature_selected.parquet',
 '../3.preprocessing_features/data/single_cell_profiles/localhost240926150001_sc_feature_selected.parquet']

### Generate dictionary with plate and data

In [4]:
# Load feature data into a dictionary, keyed on plate name without the suffix
cp_dfs = {x.split("/")[-1].split("_sc")[0]: pd.read_parquet(x) for x in fs_files}

# Print out useful information about each dataset
print(cp_dfs.keys())
[cp_dfs[x].shape for x in cp_dfs]

dict_keys(['localhost240928120001', 'localhost240927060001', 'localhost240927120001', 'localhost240926150001'])


[(12745, 641), (12397, 652), (12902, 684), (16566, 657)]

### Fit UMAP for whole plates

In [5]:
# Initialize a dictionary to store UMAP embeddings for each plate
umap_results = {}

# Fit UMAP features per dataset and save
for plate in cp_dfs:
    # Set output file for the UMAP
    output_umap_file = pathlib.Path(output_dir, f"UMAP_{plate}.parquet")

    # Check if the output file already exists
    if output_umap_file.exists():
        print(f"Skipping {output_umap_file.stem} as it already exists.")
        continue

    # Make sure to reinitialize UMAP instance per plate
    umap_fit = umap.UMAP(
        random_state=umap_random_seed, n_components=umap_n_components, n_jobs=1
    )

    # Set dataframe as the current plate
    cp_df = cp_dfs[plate]

    # Process cp_df to separate features and metadata
    cp_features = infer_cp_features(cp_df)
    meta_features = infer_cp_features(cp_df, metadata=True)

    # Subset to only failing + DMSO and healthy + DMSO for fitting (avoid flooding from the compound treated cells)
    fit_subset = cp_df[
        (
            (cp_df["Metadata_cell_type"] == "healthy")
            & (cp_df["Metadata_treatment"] == "DMSO")
        )
        | (
            (cp_df["Metadata_cell_type"] == "failing")
            & (cp_df["Metadata_treatment"] == "DMSO")
        )
    ]

    # Fit on the subset
    umap_fit.fit(fit_subset.loc[:, cp_features])

    # Transform entire dataset and convert to pandas DataFrame
    embeddings = pd.DataFrame(
        umap_fit.transform(cp_df.loc[:, cp_features]),
        columns=[f"UMAP{x}" for x in range(0, umap_n_components)],
    )
    print(f"{embeddings.shape}: {plate}")

    # Combine with metadata
    cp_umap_with_metadata_df = pd.concat(
        [cp_df.loc[:, meta_features], embeddings], axis=1
    )

    # Add treatment type column
    cp_umap_with_metadata_df["Metadata_treatment_type"] = np.select(
        [
            (cp_umap_with_metadata_df["Metadata_cell_type"] == "healthy")
            & (cp_umap_with_metadata_df["Metadata_treatment"] == "DMSO"),
            (cp_umap_with_metadata_df["Metadata_cell_type"] == "failing")
            & (cp_umap_with_metadata_df["Metadata_treatment"] == "DMSO"),
            (cp_umap_with_metadata_df["Metadata_cell_type"] == "failing")
            & (cp_umap_with_metadata_df["Metadata_treatment"] != "DMSO"),
        ],
        ["healthy + DMSO", "failing + DMSO", "failing + compound"],
        default="other",
    )
    # Add new metadata column to the list
    meta_features.append("Metadata_treatment_type")

    # Check and adjust dtypes dynamically
    for col in cp_umap_with_metadata_df.columns:
        if col in meta_features:
            # Try converting to numeric first (if possible), if not, keep as string
            try:
                cp_umap_with_metadata_df[col] = pd.to_numeric(
                    cp_umap_with_metadata_df[col], errors="raise", downcast="integer"
                )
            except ValueError:
                # If can't convert to numeric, keep as string
                cp_umap_with_metadata_df[col] = cp_umap_with_metadata_df[col].astype(
                    str
                )
        else:
            # For UMAP embeddings, ensure they're float
            cp_umap_with_metadata_df[col] = cp_umap_with_metadata_df[col].astype(float)

        # Update the 'Pathway' column for failing + DMSO and healthy + DMSO
        cp_umap_with_metadata_df["Metadata_Pathway"] = cp_umap_with_metadata_df.apply(
            lambda row: (
                "failing + DMSO"
                if row["Metadata_cell_type"] == "failing"
                and row["Metadata_treatment"] == "DMSO"
                else (
                    "healthy + DMSO"
                    if row["Metadata_cell_type"] == "healthy"
                    and row["Metadata_treatment"] == "DMSO"
                    else row["Metadata_Pathway"]
                )
            ),
            axis=1,
        )

    # Store the UMAP result in the dictionary
    umap_results[plate] = cp_umap_with_metadata_df

    # Generate output file, drop unnamed column, and save
    cp_umap_with_metadata_df.to_parquet(output_umap_file, index=False)

(12745, 2): localhost240928120001
(12397, 2): localhost240927060001
(12902, 2): localhost240927120001
(16566, 2): localhost240926150001


## Combine all plates together to generate UMAP embeddings

In [6]:
# Get common features across all plates
common_columns = set.intersection(*[set(df.columns) for df in cp_dfs.values()])

# Use the first plate to identify metadata columns (those starting with 'Metadata_')
first_df = next(iter(cp_dfs.values()))
metadata_columns = [col for col in first_df.columns if col.startswith("Metadata_")]

# Get final columns to keep: intersection of common + metadata
final_columns = list(common_columns.union(metadata_columns))

# Subset each plate's dataframe to only those columns
cp_dfs_subset = {k: df[final_columns] for k, df in cp_dfs.items()}

# Combine all feature-selected plates with common features and metadata
combined_cp_df = pd.concat(cp_dfs_subset.values(), ignore_index=True)

# Verify shape and output
print(f"Combined shape: {combined_cp_df.shape}")
combined_cp_df.head()

Combined shape: (54610, 494)


,Nuclei_Texture_InfoMeas1_PM_3_03_256,Nuclei_Texture_Contrast_Hoechst_3_00_256,Nuclei_Granularity_1_Mitochondria,Cells_Texture_InfoMeas2_ER_3_00_256,Nuclei_Texture_InverseDifferenceMoment_Hoechst_3_03_256,Nuclei_Correlation_RWC_Actin_ER,Cells_AreaShape_Zernike_4_2,Nuclei_RadialDistribution_MeanFrac_Mitochondria_1of4,Nuclei_Texture_InfoMeas2_PM_3_03_256,Cytoplasm_Texture_DifferenceEntropy_ER_3_00_256,...,Nuclei_Correlation_Overlap_ER_Hoechst,Cytoplasm_RadialDistribution_RadialCV_Mitochondria_2of4,Nuclei_AreaShape_Zernike_6_2,Nuclei_AreaShape_Zernike_7_7,Cytoplasm_Correlation_Correlation_Hoechst_PM,Cells_RadialDistribution_RadialCV_Mitochondria_1of4,Cells_Texture_InfoMeas2_ER_3_03_256,Nuclei_Intensity_MADIntensity_PM,Nuclei_Intensity_IntegratedIntensity_Hoechst,Nuclei_Intensity_MinIntensity_ER
0,-0.227471,0.186194,-0.115246,-1.829144,-0.527682,0.784299,-1.474415,4.384873,0.734586,-0.116150,...,0.788591,0.012573,-0.605354,-0.735131,-0.630170,-0.799994,-0.905620,0.119329,0.042719,0.048085
1,0.242874,-0.185889,-0.627507,0.442717,0.425134,0.189405,-0.999274,0.472266,0.376406,0.061590,...,-0.041821,-1.509353,2.296210,-0.305365,-0.142334,-1.170812,-0.316871,0.178704,-0.003797,-1.062245
2,0.661902,0.077860,0.153572,0.126137,-0.079643,-0.205251,-0.325680,-1.862054,0.322707,0.497147,...,-0.409388,0.044470,-0.853549,0.301566,-0.083048,-1.040295,-0.053428,0.404541,-0.181445,-0.151376
3,-0.282703,-0.095380,-0.466168,0.836858,-1.346657,0.720462,0.099306,0.747702,0.763409,0.514189,...,0.215614,-1.834373,1.666143,0.923572,-0.917840,-1.160285,0.028969,0.330322,0.603308,-0.630081
4,-1.024593,0.298214,-0.861445,-1.717630,-1.685917,-0.763206,-1.370080,6.298278,1.584880,3.578626,...,-0.252001,-0.874225,-0.375063,0.638295,-0.543802,-1.081872,-1.128124,2.563246,0.534686,-2.049574


In [7]:
# Process combined_cp_df to separate features and metadata
combined_cp_features = infer_cp_features(combined_cp_df)
combined_meta_features = infer_cp_features(combined_cp_df, metadata=True)

# Subset to only failing + DMSO and healthy + DMSO for fitting
combined_fit_subset = combined_cp_df[
    (
        (combined_cp_df["Metadata_cell_type"] == "healthy")
        & (combined_cp_df["Metadata_treatment"] == "DMSO")
    )
    | (
        (combined_cp_df["Metadata_cell_type"] == "failing")
        & (combined_cp_df["Metadata_treatment"] == "DMSO")
    )
]

# Initialize UMAP instance
combined_umap_fit = umap.UMAP(
    random_state=umap_random_seed, n_components=umap_n_components, n_jobs=1
)

# Fit on the subset
combined_umap_fit.fit(combined_fit_subset.loc[:, combined_cp_features])

# Transform entire dataset and convert to pandas DataFrame
combined_embeddings = pd.DataFrame(
    combined_umap_fit.transform(combined_cp_df.loc[:, combined_cp_features]),
    columns=[f"UMAP{x}" for x in range(0, umap_n_components)],
)

# Combine with metadata
combined_cp_umap_with_metadata_df = pd.concat(
    [combined_cp_df.loc[:, combined_meta_features], combined_embeddings], axis=1
)

# Add treatment type column
combined_cp_umap_with_metadata_df["Metadata_treatment_type"] = np.select(
    [
        (combined_cp_umap_with_metadata_df["Metadata_cell_type"] == "healthy")
        & (combined_cp_umap_with_metadata_df["Metadata_treatment"] == "DMSO"),
        (combined_cp_umap_with_metadata_df["Metadata_cell_type"] == "failing")
        & (combined_cp_umap_with_metadata_df["Metadata_treatment"] == "DMSO"),
        (combined_cp_umap_with_metadata_df["Metadata_cell_type"] == "failing")
        & (combined_cp_umap_with_metadata_df["Metadata_treatment"] != "DMSO"),
    ],
    ["healthy + DMSO", "failing + DMSO", "failing + compound"],
    default="other",
)

# Update the 'Pathway' column for failing + DMSO and healthy + DMSO
combined_cp_umap_with_metadata_df["Metadata_Pathway"] = combined_cp_umap_with_metadata_df.apply(
    lambda row: (
        "failing + DMSO"
        if row["Metadata_cell_type"] == "failing"
        and row["Metadata_treatment"] == "DMSO"
        else (
            "healthy + DMSO"
            if row["Metadata_cell_type"] == "healthy"
            and row["Metadata_treatment"] == "DMSO"
            else row["Metadata_Pathway"]
        )
    ),
    axis=1,
)

# Save the combined UMAP embeddings to a file
combined_output_umap_file = pathlib.Path(output_dir, "UMAP_combined.parquet")
combined_cp_umap_with_metadata_df.to_parquet(combined_output_umap_file, index=False)

print(f"Combined UMAP embeddings saved to {combined_output_umap_file}")

Combined UMAP embeddings saved to results/UMAP_combined.parquet


In [8]:
combined_cp_umap_with_metadata_df.head()

,Metadata_ImageNumber,Metadata_Cytoplasm_Parent_Cells,Metadata_heart_failure_type,Metadata_Well,Metadata_Pathway,Metadata_Site,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Number_Object_Number,Metadata_Nuclei_Location_Center_Y,Metadata_Plate,...,Metadata_Cells_Location_Center_Y,Metadata_treatment,Metadata_Cells_Location_Center_X,Metadata_cell_type,Metadata_WellCol,Metadata_Image_Count_Cells,Metadata_Cytoplasm_Parent_Nuclei,UMAP0,UMAP1,Metadata_treatment_type
0,2,1,None,B02,healthy + DMSO,f04,855.599578,6,277.977481,localhost240928120001,...,268.256144,DMSO,809.932735,healthy,2,9,6,13.979544,-1.069638,healthy + DMSO
1,3,1,None,B02,healthy + DMSO,f10,613.937003,3,209.544343,localhost240928120001,...,204.845356,DMSO,640.790427,healthy,2,4,3,13.065675,0.019595,healthy + DMSO
2,5,1,None,B02,healthy + DMSO,f14,916.379710,3,280.023188,localhost240928120001,...,333.104966,DMSO,927.500461,healthy,2,9,3,14.179733,0.349703,healthy + DMSO
3,7,1,None,B02,healthy + DMSO,f18,530.411492,3,80.037145,localhost240928120001,...,88.534307,DMSO,515.843583,healthy,2,18,3,13.331791,0.886280,healthy + DMSO
4,1,1,None,B02,healthy + DMSO,f00,958.301695,4,69.265085,localhost240928120001,...,96.955249,DMSO,955.244723,healthy,2,10,4,11.598928,0.409980,healthy + DMSO
